# Dash Testing

In [ ]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import matplotlib as plt
from matplotlib.pyplot import figure

import sheworewhat as sww

# alt.data_transformers.disable_max_rows()

closet = sww.closet_df()
worn_df = sww.complete_df(closet)
top_10, heat_df = sww.top_10_df()

def plot_mostworn():
    most_worn = worn_df.nlargest(15, columns="Count")
    closet_comp = alt.Chart(most_worn, title="2023 Most Worn Pieces"
                       ).mark_bar(color="#6ce4d8"
                         ).encode(alt.X("Name",
                                        title="",
                                        axis=alt.Axis(labelAngle=-45), 
                                        sort="-y"),
                                  alt.Y("Count", 
                                        title="Times Worn",
                                        axis=alt.Axis(tickMinStep=1)),
                                  alt.Tooltip("Count")
                                 ).configure_axis(grid=False, domain=False)
                
    return closet_comp.to_html()

def plot_facet():
    
    categories = ["Top", "Bottom", "Full Body", "Outerwear", "Accessory", "Shoes"]

    cat_plots = []

    for i in categories:
        category_worn = worn_df.loc[worn_df["Category"] == i].nlargest(5, columns="Count")

        category_plot = alt.Chart(category_worn, title=f"2023 Most Worn {i}"
                           ).mark_bar(color="#827191",
                           ).encode(alt.X("Name", title="", axis=alt.Axis(labelAngle=-45), sort="-y"),
                                    alt.Y("Count", 
                                    title="# of Times Worn",
                                    axis=alt.Axis(tickMinStep=1)),
                                    alt.Tooltip(["Name", "Count"])
                            ).properties(
                                height=200,
                                width=150
                                )
        cat_plots.append(category_plot)

    # configure altair charts
    row1 = alt.hconcat(cat_plots[0], cat_plots[1], cat_plots[2])
    row2 = alt.hconcat(cat_plots[3], cat_plots[4], cat_plots[5])

    category_plot = alt.vconcat(row1, row2).configure_axis(grid=False, domain=False)
    return category_plot.to_html()

def plot_heatmap(top_10, df, i=0):
    """
    Function for heatmap plot.
    
    Parameters:
    ----------- 
         top_10 : list
            List containing the IDs of the top 10 most worn items.
            
         df : pandas.DataFrame  
         
    Returns:
    --------
        heatplot : altair.Chart
            Heatmap plot for a single item over a single calender year. 
             
    """
    
    # column of day of week for one calender year 
    time_df = pd.DataFrame()
    time_df["Date"] = pd.date_range(df["Date"].min(), periods=365) 
    time_df["Day"] = time_df["Date"].dt.day_name()
    
    
    heatmap_data = df.loc[df["ID"] == top_10[i]] # need to make this dynamic in plot
    item_name = heatmap_data["Brand"].iloc[0] + " " + heatmap_data["Item"].iloc[0]
    
    # isolate item data
    year = pd.merge(time_df, heatmap_data, how="outer", on="Date")
    year["Item"] = year["Item"].replace(np.nan, 0) 
    year["Bool"] = np.where(year["Item"] == 0, 0, 1)

    # this is horrible to read lol
    # wrangling for prettier plotting 
    week = time_df["Date"].dt.isocalendar()
    year["Week"] = week["week"].fillna(52)
    year["Week"] = year["Week"].fillna(52)
    year['First_day'] = year['Date'] - year['Date'].dt.weekday * np.timedelta64(1, 'D')
    week = time_df["Date"].dt.strftime('%m-%d-%y')
    year["Week"] = year["First_day"].dt.strftime('%m-%d')
    year = year[["Date", "Day", "Item", "ID", "Bool", "Week"]]

    weekdays = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Sunday"]

    heat_plot = alt.Chart(year, title=f"{item_name} in 2023").mark_rect(
        stroke="white",
        strokeWidth=3,
        opacity=0.9,
        ).encode(
        alt.X("Week:O", 
              axis = alt.Axis(labelAngle=-60, )
             ), 
        alt.Y("Day", 
              sort=weekdays,
             title=""),
        alt.Color("Bool", 
                scale=alt.Scale(
                    domain=[0,1], 
                    range=["#e0ddd5", "#74a675"]),
                    legend=None
                 ),
        alt.Tooltip(["Date", "Day"])
        ).properties(
        height=200,
        width=600
        ).configure_axis(grid=False, domain=False)
    return heat_plot.to_html()

app = Dash(__name__)
server = app.server

app.title = "She Wore What 2023"

app.layout = dbc.Container(
    [
        html.Br(),
        dbc.Row(
            dbc.Col(
                dbc.Card(
                    dbc.CardBody(
                        html.B("She Wore What 2023"),
                    ), 
                ),
                style={"font-weight": "bold", 
                       "color": "#000000", 
                       "font-family" :"Garamond",
                      "font-size" : "120%"},
                className='text-center mt-4 mb-5'
            )
        ),
        dbc.Row([
            dbc.Col(
                dbc.Card([
                    dbc.CardBody([
                        html.Div([
                            html.Iframe(
                                id='mostworn',
                                style={'border-width': '0', 'width': '100%', 'height': '600px'},
                                srcDoc=plot_heatmap(top_10, heat_df)
                                )
                        ]),
                        html.Div([
                            html.Iframe(
                                id='mostworn2',
                                style={'border-width': '0', 'width': '100%', 'height': '600px'},
                                srcDoc=plot_facet()
                                )
                        ]),
                    ]),
                    
                ])
            ), 
            dbc.Col(
                dbc.Card([
                    dbc.CardBody([
                        html.Div([
                            html.Iframe(
                                id='mostworn3',
                                style={'border-width': '0', 'width': '100%', 'height': '600px'},
                                srcDoc=plot_mostworn()
                                )
                        ]),
                    ])
                ])
            ),
        ]),
        dbc.Col(
            dbc.Row(
                dbc.Card(
                    dbc.CardBody([
                        html.B("Title2")
                    ]),
                )
            ),
        )
    ])

if __name__ == '__main__':
    app.run_server(
        port = 8073,
        debug=False)

In [3]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import matplotlib as plt
from matplotlib.pyplot import figure

import sheworewhat as sww

closet = sww.closet_df()
worn_df = sww.complete_df(closet)
top_10, heat_df = sww.top_10_df()

app = Dash(__name__)
server = app.server

app.title = "She Wore What 2023"

app.layout = dbc.Container(
    [
        html.Br(),
        dbc.Row(
            dbc.Col(
                dbc.Card(
                    dbc.CardBody(
                        html.B("She Wore What 2023"),
                    ), 
                ),
                style={"font-weight": "bold", 
                       "color": "#000000", 
                       "font-family" :"Garamond",
                      "font-size" : "120%"},
                className='text-center mt-4 mb-5'
            )
        ),
        dbc.Row([
            dbc.Col(
                dbc.Card([
                    dbc.CardBody([
                        
                        html.H4("Card title", className="card-title"),
                            html.P(
                            "Some quick example text to build on the card title and "
                            "make up the bulk of the card's content.",
                            className="card-text",
                        ),
                    dbc.Button("Go somewhere", color="primary"),
                        html.Div([
                            html.Iframe(
                                id='mostworn',
                                style={'border-width': '0', 'width': '100%', 'height': '600px'},
                                srcDoc= sww.plot_heatmap(top_10, heat_df).to_html()
                            )
                        ]),
                        dbc.CardFooter("This is the footer")
                    ]),    
                ])
            ), 
            dbc.Col(
                dbc.Card([
                    dbc.CardBody([
                        html.Div([
                            html.Iframe(
                                id='mostworn3',
                                style={'border-width': '0', 'width': '100%', 'height': '600px'},
                                srcDoc = sww.plot_mostworn(worn_df).to_html()
                                )
                        ]),
                    ])
                ])
            ),
        ]),
    ])

if __name__ == '__main__':
    app.run_server(
        port = 8076,
        debug=False)

Dash is running on http://127.0.0.1:8076/

Dash is running on http://127.0.0.1:8076/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8076
Press CTRL+C to quit
127.0.0.1 - - [02/Jan/2023 19:52:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 19:52:52] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_7_1m1671594179.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 19:52:52] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_7_1m1671594179.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 19:52:52] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_7_1m1671594179.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 19:52:52] "GET /_dash-component-suites/dash/deps/react@16.v2_7_1m1671594179.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 19:52:52] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_2_1m1671594223.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 19:52:52] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_7_1m1671594179.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 19:52:52] "GET /_d

In [2]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import matplotlib as plt
from matplotlib.pyplot import figure

import sheworewhat as sww

closet = sww.closet_df()
worn_df = sww.complete_df(closet)
top_10, heat_df = sww.top_10_df()

app = Dash(__name__)
server = app.server

app.title = "She Wore What 2023"

app.layout = dbc.Container(
    [
        html.Br(),
        dbc.Row(
            dbc.Col(
                dbc.Card(
                    dbc.CardBody(
                        html.B("She Wore What 2023"),
                    ), 
                ),
                style={"font-weight": "bold", 
                       "color": "#000000", 
                       "font-family" :"Garamond",
                      "font-size" : "120%"},
                className='text-center mt-4 mb-5'
            )
        ),
        dbc.Row([
            dbc.Col(
                dbc.Card([
                    dbc.CardBody([
                        html.H4("Card title", className="card-title"),
                            html.P(
                            "Some quick example text to build on the card title and "
                            "make up the bulk of the card's content.",
                            className="card-text",
                        ),
                        dbc.Button("A Button", color="#e0ddd5"),
                            html.Div([
                                html.Iframe(
                                    id='mostworn',
                                    style={'border-width': '0', 'width': '100%', 'height': '600px'},
                                    srcDoc= sww.plot_heatmap(top_10, heat_df).to_html()
                                )
                            ]),
                            html.Div([
                                html.Iframe(
                                    id='mostworn2',
                                    style={'border-width': '0', 'width': '100%', 'height': '600px'},
                                    srcDoc= sww.plot_cpw(worn_df).to_html()
                                )
                            ]),
                        dbc.CardFooter("This is the footer")
                    ]),    
                ])
            ), 
            dbc.Col(
                dbc.Card([
                    dbc.CardBody([
                        html.Div([
                            html.Iframe(
                                id='mostworn3',
                                style={'border-width': '0', 'width': '100%', 'height': '600px'},
                                srcDoc = sww.plot_mostworn(worn_df).to_html()
                                )
                        ]),
                    ])
                ])
            ),
        ]),
    ],
    fluid=True
)

if __name__ == '__main__':
    app.run_server(
        port = 8071,
        debug=False)

Dash is running on http://127.0.0.1:8071/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8071
Press CTRL+C to quit
127.0.0.1 - - [02/Jan/2023 19:59:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 19:59:52] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 19:59:52] "GET /_dash-dependencies HTTP/1.1" 200 -


In [1]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import matplotlib as plt
from matplotlib.pyplot import figure

import sheworewhat as sww

closet = sww.closet_df()
worn_df = sww.complete_df(closet)
top_10, heat_df = sww.top_10_df()

app = Dash(__name__)
server = app.server

app.title = "She Wore What 2023"
app = Dash(__name__, external_stylesheets=[dbc.themes.LUX])
app.layout = dbc.Container(
    [
        html.Br(),
        dbc.Row(
            dbc.Col(
                html.B("She Wore What 2023"),
                    style={"font-weight": "bold", 
                       "color": "#000000", 
                      "font-size" : "150%"}
            )
        ),
        html.Br(),
        dbc.Row([
            dbc.Col([
                    html.P(
                    "Hi! My name is Jasmine and I tracked every single item of clothing I wore in 2023. "
                    "As a data scientist and sustainable fashion enthusiast, I thought "
                    "this would be a fun project to help me analyze my personal style"
                    "trends so I can make smarter decisions about my purchases in the future.",
                    )
            ], width={"size": 6})
        ]),
        dbc.Row([
            dbc.Col([
                    html.Div([
                        html.Iframe(
                            id='top10',
                            style={'border-width': '0', 'width': '100%', 'height': '600px'},
                            srcDoc= sww.plot_mostworn(worn_df).to_html()
                        ),
                    ]),
            ]),
            dbc.Col([
                    html.P(
                    "To the left, we can see my top 10 most worn items. Some of these seem"
                    " obvious -- my Hollister puffer coat is my go-to winter coat. My gold brand "
                    "hoop earrings are a daily staple and part of my personal brand. But this plot"
                    " really doesn't say much -- it might be better to break down the items by category."
                    "Note to self: would be cool to insert images here", 
                    )
            ])
        ]),
        dbc.Row([
            dbc.Col([
                html.Div([
                    html.Iframe(
                        id='mostworn3',
                        style={'border-width': '0', 'width': '100%', 'height': '600px'},
                        srcDoc = sww.plot_heatmap(top_10, heat_df).to_html()
                        )
                ]),
            ]),
        ])
        ],
    fluid=True
)

if __name__ == '__main__':
    app.run_server(
        port = 8077,
        debug=False)

Dash is running on http://127.0.0.1:8078/

Dash is running on http://127.0.0.1:8078/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8078
Press CTRL+C to quit
127.0.0.1 - - [02/Jan/2023 21:57:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 21:57:27] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_7_1m1671594179.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 21:57:27] "GET /_dash-component-suites/dash/deps/react@16.v2_7_1m1671594179.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 21:57:27] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_7_1m1671594179.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 21:57:27] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_7_1m1671594179.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 21:57:27] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_2_1m1671594223.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 21:57:27] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_7_1m1671594179.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 21:57

In [2]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np


import sheworewhat as sww

closet = sww.closet_df()
worn_df = sww.complete_df(closet)
top_10, heat_df = sww.top_10_df()

app = Dash(__name__)
server = app.server

app.title = "She Wore What 2023"

app = Dash(__name__, external_stylesheets=[dbc.themes.LUX]) #QUARTZ OR LUX
app.layout = dbc.Container(
    [
        html.Br(),
        dbc.Row(
            dbc.Col(
                html.B("She Wore What 2023"),
                    style={"font-weight": "bold", 
                       "color": "#000000", 
                      "font-size" : "150%"}
            )
        ),
        html.Br(),
        dbc.Row([
            dbc.Col([
                    html.P(
                    "Hi! My name is Jasmine and I tracked every single item of clothing I wore in 2023. "
                    "As a data scientist and sustainable fashion enthusiast, I thought "
                    "this would be a fun project to help me analyze my personal style"
                    "trends so I can make smarter decisions about my purchases in the future.",
                    )
            ], width={"size": 6})
        ]),
        dbc.Row([
            dbc.Col([ #
                html.Div(
                    dbc.Accordion(
                        [
                            # top 10 plots + facet plots
                            dbc.AccordionItem([
                                dbc.Row([
                                    dbc.Col([
                                        html.Div([
                                            html.Iframe(
                                                id='top10',
                                                style={'border-width': '0', 
                                                       'width': '100%', 'height': '400px'},
                                                srcDoc= sww.plot_mostworn(worn_df).to_html()
                                            ),
                                        ]),
                                    ]),
                                    dbc.Col([
                                        html.Br(), 
                                        html.Br(),
                                        html.P("My top 10 most worn items of the year. Some of these seem"
                                            " obvious -- my Hollister puffer coat is my go-to winter coat. My gold brand "
                                            "hoop earrings are a daily staple. But this plot doesn't tell us much on it's own"
                                            "let's break down the items by category. "
                                            "Note to self: would be cool to insert images here"
                                        )
                                    ]),
                                ]),
                                dbc.Row([
                                    dbc.Col([
                                        html.Br(),
                                        html.P("Facets!")
                                    ]),
                                    dbc.Col([
                                       html.Div([
                                            html.Iframe(
                                                id='top5_cat',
                                                style={'border-width': '0', 
                                                       'width': '100%', 'height': '800px'},
                                                srcDoc= sww.plot_facet(worn_df).to_html()
                                            ),
                                        ]), 
                                    ])
                                ]),
                            ],
                                title="Most Worn Items Overall",
                            ),
                            dbc.AccordionItem(
                                [
                                dbc.Row([
                                    dbc.Col([
                                        html.Br(),
                                        html.Br(),
                                        html.P("Fun little heatmap of the top 10 items -- need to work on drop down menu."
                                        )
                                    ]),
                                    dbc.Col([
                                        html.Div([
                                            html.Iframe(
                                                id='heatmap',
                                                style={'border-width': '0', 'width': '100%', 'height': '300px'},
                                                srcDoc = sww.plot_heatmap(top_10, heat_df).to_html()
                                                )
                                        ]),  
                                    ]),
                                ]),
                                ], title="Item Heatmap Throughout 2023"
                            ),
                            # cost per wear
                            dbc.AccordionItem(
                                [
                                    dbc.Row([
                                        dbc.Col([
                                            html.P("Cost-per-wear: price of item / number of time worn" 
                                                   "Please note, the cost-per-wear was only calculated for items for which price"
                                                   "was known, including items purchased secondhand. ")
                                        ]),
                                        dbc.Col([
                                            html.Div([
                                                html.Iframe(
                                                    id='costperwear',
                                                    style={'border-width': '0', 'width': '100%', 'height': '400px'},
                                                    srcDoc = sww.plot_cpw(worn_df).to_html()
                                                )
                                            ]),
                                        ]),
                                    ])
                                ],
                                title="Cost Per Wear",
                            ),
                            dbc.AccordionItem(
                                "This is the content of the third section",
                                title="Item 3",
                            ),
                        ], start_collapsed=True
                    )
                )    
            ]), 
        ])
        ],
    fluid=True
)

if __name__ == '__main__':
    app.run_server(
        port = 8078,
        debug=False)


Dash is running on http://127.0.0.1:8078/

Dash is running on http://127.0.0.1:8078/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8078
Press CTRL+C to quit
127.0.0.1 - - [02/Jan/2023 23:55:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 23:55:58] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2023 23:55:58] "GET /_dash-dependencies HTTP/1.1" 200 -


Rearrange the accordion / omit boring plots

In [10]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np


import sheworewhat as sww

closet = sww.closet_df()
worn_df = sww.complete_df(closet)
top_10, heat_df = sww.top_10_df()

app = Dash(__name__)
server = app.server

app.title = "She Wore What 2023"

app = Dash(__name__, external_stylesheets=[dbc.themes.LUX]) #QUARTZ OR LUX
app.layout = dbc.Container(
    [
        html.Br(),
        dbc.Row(
            dbc.Col(
                html.B("She Wore What 2023"),
                    style={"font-weight": "bold", 
                       "color": "#000000", 
                      "font-size" : "150%"}
            )
        ),
        html.Br(),
        dbc.Row([
            dbc.Col([
                    html.P(
                    "Hi! My name is Jasmine and I tracked every single item of clothing I wore in 2023. "
                    "As a data scientist and sustainable fashion enthusiast, I thought "
                    "this would be a fun project to help me analyze my personal style"
                    "trends so I can make smarter decisions about my purchases in the future.",
                    )
            ], width={"size": 6})
        ]),
        dbc.Row([
            dbc.Col([ 
                html.Div(
                    dbc.Accordion(
                        [
                            dbc.AccordionItem(
                                [
                                    dbc.Row([
                                        dbc.Col([
                                            html.Br(),
                                            html.Br(),
                                            html.P("The first thing I did was log everything in my closet. "
                                                   "A tedious process, but in the end I had [var] pieces."
                                                   "The majority of my closet is black or white."
                                                   "Colors represented in my closet -- need to add secondary colors.")
                                        ], width={"size": 6}
                                        ),
                                         dbc.Col([
                                           html.Div([
                                                html.Iframe(
                                                    id="color",
                                                    style={'border-width': '0', 'width': '100%', 'height': '400px'},
                                                    srcDoc = sww.plot_color(worn_df).to_html()
                                                )
                                            ])
                                        ]),
                                    ]),
                                    dbc.Row([
                                        dbc.Col([
                                            html.Br(),
                                            html.Br(),
                                            html.P("I own a lot of tops and very few coats.")
                                        ]),
                                        dbc.Col([
                                            html.Br(),
                                            html.Div([
                                                html.Iframe(
                                                    id="catwheel",
                                                    style={'border-width': '0', 'width': '100%', 'height': '400px'},
                                                    srcDoc = sww.plot_categories(worn_df).to_html()
                                                )
                                            ])
                                        ]),
                                        dbc.Col([
                                            html.Br(),
                                            html.Br(),
                                            html.Br(),
                                           html.Div([
                                                html.Iframe(
                                                    id="bought",
                                                    style={'border-width': '0', 'width': '100%', 'height': '400px'},
                                                    srcDoc = sww.plot_bought(worn_df).to_html()
                                                )
                                            ])
                                        ])
                                    ]),
                                ],
                                title = "Wardrobe Analysis"
                            ),
                            dbc.AccordionItem(
                                [
                                dbc.Row([
                                    dbc.Col([
                                        html.P("My top 10 most worn items ..."
                                        )
                                    ]),
                                ]),
                                dbc.Row([
                                    dbc.Col([
                                        html.Div([
                                            html.Iframe(
                                                id='top10',
                                                style={'border-width': '0', 'width': '100%', 'height': '300px'},
                                                srcDoc = sww.plot_mostworn(worn_df).to_html()
                                                )
                                        ]),
                                    ], width={"size": 5}),
                                    dbc.Col([
                                        html.Div([
                                            html.Iframe(
                                                id='heatmap',
                                                style={'border-width': '0', 'width': '100%', 'height': '300px'},
                                                srcDoc = sww.plot_heatmap(top_10, heat_df).to_html()
                                                )
                                        ]),  
                                    ]),
                                ]),
                                ], title="Most Worn Items of 2023"
                            ),
                            # cost per wear
                            dbc.AccordionItem(
                                [
                                    dbc.Row([
                                        dbc.Col([
                                            html.P("Cost-per-wear: price of item / number of time worn" 
                                                   "Please note, the cost-per-wear was only calculated for items for which price"
                                                   "was known, including items purchased secondhand. ")
                                        ]),
                                        dbc.Col([
                                            html.Div([
                                                html.Iframe(
                                                    id='costperwear',
                                                    style={'border-width': '0', 'width': '100%', 'height': '425px'},
                                                    srcDoc = sww.plot_cpw(worn_df).to_html()
                                                )
                                            ]),
                                        ]),
                                    ])
                                ],
                                title="Cost Per Wear",
                            ),
                            dbc.AccordionItem(
                                "This is the content of the third section",
                                title="Item 3",
                            ),
                        ], start_collapsed=True, color="red"
                    )
                )    
            ]), 
        ])
        ],
    fluid=True
)

if __name__ == '__main__':
    app.run_server(
        port = 8077,
        debug=False)

TypeError: The `dash_bootstrap_components.Accordion` component (version 1.2.1) received an unexpected keyword argument: `color`
Allowed arguments: active_item, always_open, children, className, class_name, flush, id, key, loading_state, persisted_props, persistence, persistence_type, start_collapsed, style